[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OlivierGeorgeon/Developmental-AI-Lab/blob/master/docs/agent3.ipynb)

# THE AGENT WHO TAMED THE TURTLE

# Learning objectives

Upon completing this lab, you will be able to assign appropriate valences to interactions, enabling a developmental agent to exhibit exploratory behavior in a simulated environment.

# Setup
## Import the turtle environment

In [66]:
!pip3 install ColabTurtle
from ColabTurtle.Turtle import *

## Define the Agent class

In [67]:
class Agent:
    def __init__(self, _valences):
        """ Creating our agent """
        self._valences = _valences
        self._action = None
        self._predicted_outcome = None

    def action(self, _outcome):
        """ tracing the previous cycle """
        if self._action is not None:
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {_outcome}, " 
                  f"Prediction: {self._predicted_outcome == _outcome}, Valence: {self._valences[self._action][_outcome]}")

        """ Computing the next action to enact """
        # TODO: Implement the agent's decision mechanism
        self._action = 0
        # TODO: Implement the agent's anticipation mechanism
        self._predicted_outcome = 0
        return self._action


## Define the tutle environment class

You don't need to worry about the code of the ColabTurtleEnvironment below.

Just know that this environment:
* interprets the agent's actions as follows  `0`: move forward, `1`: turn left, `2`: turn right.
* returns outcome `1` when the turtle bumps into the border of the window, and `0` otherwise.

In [68]:
# @title Initialize the turtle environment

BORDER_WIDTH = 20

class ColabTurtleEnvironment:

    def __init__(self):
        """ Creating the Turtle window """
        bgcolor("lightGray")
        penup()
        goto(window_width() / 2, window_height()/2)
        face(0)
        pendown()
        color("green")

    def outcome(self, action):
        """ Enacting an action and returning the outcome """
        _outcome = 0
        for i in range(10):
            # _outcome = 0
            if action == 0:
                # move forward
                forward(10)
            elif action == 1:
                # rotate left
                left(4)
                forward(2)
            elif action == 2:
                # rotate right
                right(4)
                forward(2)

            # Bump on screen edge and return outcome 1
            if xcor() < BORDER_WIDTH:
                goto(BORDER_WIDTH, ycor())
                _outcome = 1
            if xcor() > window_width() - BORDER_WIDTH:
                goto(window_width() - BORDER_WIDTH, ycor())
                _outcome = 1
            if ycor() < BORDER_WIDTH:
                goto(xcor(), BORDER_WIDTH)
                _outcome = 1
            if ycor() > window_height() - BORDER_WIDTH:
                goto(xcor(), window_height() -BORDER_WIDTH)
                _outcome = 1

            # Change color
            if _outcome == 0:
                color("green")
            else:
                # Finit l'interaction
                color("red")
                # if action == 0:
                #     break
                if action == 1:
                    for j in range(10):
                        left(4)
                elif action == 2:
                    for j in range(10):
                        right(4)
                break

        return _outcome

## Define the valence of interactions

In [69]:
valences = [[1, -1], 
            [-1, -1],
            [-1, -1]]

The valence table specifies the valence of each interaction. An interaction is a tuple (action, outcome):

|| 0 Not bump | 1 Bump|
|---|---|---|
| 0 Forward| 1 | -1 |
| 1 Left | -1 | -1 |
| 2 Right| -1 | -1 |

## Instantiate the agent

In [70]:
a = Agent(valences)

## Run the simulation 

In [71]:
# @title Run the simulation

initializeTurtle()

# Parameterize the rendering
bgcolor("lightGray")
penup()
goto(window_width() / 2, window_height()/2)
face(0)
pendown()
color("green")
speed(10)

e = ColabTurtleEnvironment()

outcome = 0
for i in range(10):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: 1
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: 1
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: 1
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1


Observe the turtle moving in a straigt line until it bumps into the border of the window

# PRELIMINARY EXERCISE

Copy Agent2 that you designed in your previous assignment to this notebook. 

Observe how your Agent2 behaves in this environment 

# ASSIGNMENT

Implement Agent3 by modifying your previous Agent2 such that it can select 3 possible actions: `0`, `1`, or `2`.

Choose the valences of interactions so that the agent does not remain stuck in a corner of the environment. 

## Create Agent3 by overriding the class Agent or your previous class Agent2

In [79]:
import random

class Agent3(Agent):
    def __init__(self, _valences):
        """Creating our turtle-taming agent"""
        super().__init__(_valences)
        # Mémoire pour stocker le dernier outcome observé pour chaque action
        self.memory = {}
        # Compteur de prédictions correctes consécutives
        self.correct_count = 0
        # Seuil d'ennui (fixe ici, mais tu peux le rendre progressif si besoin)
        self.boredom_threshold = 4
        # Pour mémoriser l'action qui a provoqué l'ennui
        self.last_bored_action = None

    def action(self, _outcome):
        """Tracing the previous cycle"""
        if self._action is not None:
            # Met à jour la mémoire avec le dernier outcome observé
            self.memory[self._action] = _outcome

            # Vérifie si la prédiction était correcte
            satisfied = (self._predicted_outcome == _outcome)

            # Met à jour le compteur de prédictions correctes
            if satisfied:
                self.correct_count += 1
            else:
                self.correct_count = 0

            # Calcule la valence
            valence = self._valences[self._action][_outcome]

            # Vérifie si l'agent s'ennuie
            bored = (self.correct_count >= self.boredom_threshold)

            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, "
                  f"Outcome: {_outcome}, Prediction: {satisfied}, "
                  f"Valence: {valence}, Bored: {bored}")
        else:
            bored = False

        """Decision mechanism"""
        if bored:
            # Sauvegarde de l'action ennuyeuse
            self.last_bored_action = self._action

            # Sélectionne aléatoirement une autre action que celle ennuyeuse
            possible_actions = [0, 1, 2]
            if self.last_bored_action in possible_actions:
                possible_actions.remove(self.last_bored_action)
            self._action = random.choice(possible_actions)
            self.correct_count = 0

        else:
            # Sélection des actions avec la meilleure valence anticipée
            best_valence = -float('inf')
            best_actions = []

            for action in [0, 1, 2]:
                if action in self.memory:
                    predicted_outcome = self.memory[action]
                    predicted_valence = self._valences[action][predicted_outcome]

                    if predicted_valence > best_valence:
                        best_valence = predicted_valence
                        best_actions = [action]
                    elif predicted_valence == best_valence:
                        best_actions.append(action)

            # Tirage aléatoire équitable si plusieurs actions ont la même valence max
            if best_actions:
                self._action = random.choice(best_actions)
            elif self._action is None:
                self._action = 0

        """Anticipation mechanism"""
        if self._action in self.memory:
            self._predicted_outcome = self.memory[self._action]
        else:
            self._predicted_outcome = 0

        return self._action

## Choose the valence table

Replace the `valences` table by your choice in the code below

In [80]:
valences = [[1, -1], 
            [-1, 1],
            [-1, 1]]

## Test your agent in the TurtleEnvironment

In [82]:
initializeTurtle()

# Parameterize the rendering
bgcolor("lightGray")
penup()
goto(window_width() / 2, window_height()/2)
face(0)
pendown()
color("green")
speed(10)

a = Agent3(valences)
e = ColabTurtleEnvironment()

outcome = 0
for i in range(50):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: 1, Bored: False
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: -1, Bored: True
Action: 2, Prediction: 0, Outcome: 1, Prediction: False, Valence: 1, Bored: False
Action: 2, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: False
Action: 2, Prediction: 1, Outcome: 0, Prediction: False, Valence: -1, Bored: False
Action: 2, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Predicti

## Improve your agent's code

If your agent gets stuck against a border or in a corner, modify the valences or the code. 
Try different ways to handle boredome or to select random actions. 
In the next lab, you will see how to design an agent that can adapt to the context.

In [87]:

valences = [
    [ -1, -1],
    [ -1,  1],
    [ -1, 1]
]

initializeTurtle()
# Parameterize the rendering
bgcolor("lightGray")
penup()
goto(window_width() / 2, window_height()/2)
face(0)
pendown()
color("green")
speed(10)

a = Agent3(valences)
e = ColabTurtleEnvironment()

outcome = 0
for i in range(50):
    action = a.action(outcome)
    outcome = e.outcome(action)

Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 0, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 0, Outcome: 1, Prediction: False, Valence: -1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: -1, Bored: False
Action: 0, Prediction: 1, Outcome: 1, Prediction: True, Valence: -1, Bored: True
Action: 2, Prediction: 0, Outcome: 1, Prediction: False, Valence: 1, Bored: False
Action: 2, Prediction: 1, Outcome: 1, Prediction: True, Valence: 1, Bored: False
Action: 2, Prediction: 1, Outcome: 0, Prediction: False, Valence: -1, Bored: False
Action: 0, Prediction: 1, Outcome: 0, Prediction: False, Valence: -1, Bored: False
Action: 2, Pred

## Report 

Explain what you programmed and what results you observed. Export this document as PDF including your code, the traces you obtained, and your explanations below (no more than a few paragraphs):

# Rapport — Agent 3

## Exercice préliminaire : Agent2 dans TurtlePy

Avec l'agent2, la tortue ne peut qu'avancer ou tourner à gauche.
Elle ne peut jamais tourner à droite, donc elle a moins de possibilités de mouvement.
Quand elle touche un mur, elle tourne toujours dans le même sens et à plus de chance de rester bloquée dans un coin.

## Objectif


L’objectif principal de cette partie était d’améliorer Agent2 pour l’adapter à l’environnement TurtlePy afin qu’il évite au maximum de rester bloqué contre les murs.

Pour cela, nous avons implémenté Agent3 en étendant Agent2 afin de gérer trois actions au lieu de deux :

- Action 0 : Avancer
- Action 1 : Tourner à gauche
- Action 2 : Tourner à droite

L’agent utilise les mêmes mécanismes qu’Agent2 :
- Mémoire des outcomes,
- Anticipation à partir de cette mémoire,
- Décision basée sur la valence (choisit l’action à valence anticipée maximale),
- Gestion de l’ennui (changement d’action après 4 prédictions correctes consécutives).

##  Améliorations et tentatives d’optimisation dans le code

### Tirage aléatoire en cas d’égalité de valence

Si plusieurs actions ont la même valence maximale, l’agent effectue désormais un tirage aléatoire équitable parmi elles.
Cette amélioration empêche l’agent de sélectionner systématiquement la première action rencontrée et de rester bloqué dessus lorsqu’elle a la même valence que d’autres. Cela rend son comportement plus flexible et lui permet d’explorer davantage l’environnement.

### Test de variation de l’ennui avec epsilon

Nous avons également testé une variation de l’ennui avec une variable epsilon, afin de rendre le comportement de l’agent plus aléatoire et dynamique.
Cependant, les résultats n’étaient pas concluants et cette approche a été abandonnée dans la version finale de l’Agent3.

## Analyse des comportements observés

### Cas 1 — (`valences = [[ 1, -1],[ -1, 1],[ -1, 1]]`)

Dans cette configuration, les actions `1` et `2` (tourner à gauche ou à droite) offrent la même valence positive +1 lorsqu’elles sont utilisées après une collision. Cela donne à l’agent la possibilité de tourner dès qu’il tape un mur.

Au début, l’agent n’a aucune information sur les actions disponibles. Il commence naturellement par l’action `0` (avancer), qui lui donne une valence positive tant qu’il ne rencontre pas d’obstacle. Lorsque la tortue se cogne contre le bord, la valence de l’action `0` devient négative, mais comme les actions `1` et `2` n’ont pas encore été essayées, l’agent continue à avancer. Il reste donc bloqué contre le mur jusqu’à atteindre le seuil d’ennui.

Une fois ce seuil atteint, l’agent choisit aléatoirement entre les actions `1` et `2` dans notre cas `1`. Cela lui permet de découvrir une nouvelle action de rotation et de se dégager de l’obstacle. Par la suite, il alterne entre avancer (`0`) et tourner (`1`) selon la situation rencontrée. Grâce au même mécanisme d’ennui, il finit aussi par tester la deuxième action de rotation.

Le fait que les actions `1` et `2` aient la même valence maximale permet au tirage aléatoire d’éviter que l’agent reste fixé sur une seule direction lorsque qu'il rentre en collision avec un mur. Ce comportement rend la navigation moins rigide et améliore sa capacité à éviter de rester bloqué contre les murs.


### Cas 2 — (`valences = [[ -1, -1],[ -1, 1],[ -1, 1]]`)

Dans ce scénario, l’action `0` (avancer) a une valence négative dans toutes les situations, que ce soit en avançant librement ou en heurtant un mur. En revanche, les actions `1` et `2` (tourner à gauche et à droite) ont une valence positive lorsqu’elles sont utilisées après une collision.

Au début, l’agent ne connaît aucune action et commence donc naturellement par l’action `0`. Comme cette action est toujours associée à une valence négative, il accumule rapidement des interactions négatives. Malgré cela, il persiste sur cette action puisqu’il n’a encore rien appris sur les autres.

Lorsque le seuil d’ennui est atteint, il explore aléatoirement une autre action, soit `1` soit `2` dans notre cas `2`. C’est à ce moment qu’il découvre une valence positive lorsqu’il tourne après une collision, ce qui en fait une meilleure option que l’action `0` dans ces situations. Par la suite, grâce à de nouveaux épisodes d’ennui, il finit par découvrir la deuxième action de rotation restante `1`.

Une fois qu’il a mémorisé les trois actions, son comportement devient plus varié. En dehors des collisions, toutes les actions ont une valence égale à `-1`, ce qui entraîne un tirage aléatoire entre elles. Cela permet d’éviter un comportement déterministe et rend sa trajectoire plus vivante et moins linéaire.

De plus, grâce aux valences positives des actions gauche et droite lors des collisions, l’agent est capable de se débloquer efficacement lorsqu’il rencontre un mur. Il peut ainsi continuer à explorer l’environnement tout en conservant des trajectoires dynamiques et variées.



## Conclusion

En résumé, l’agent commence avec une stratégie simple consistant à avancer. Grâce au mécanisme d’ennui et au tirage aléatoire, il découvre progressivement les actions de rotation, ce qui lui permet d’adopter un comportement plus flexible et mieux adapté à l’environnement TurtlePy.

La première configuration de valences (`[[ 1, -1],[ -1, 1],[ -1, 1]]`) est efficace. Elle encourage l’agent à avancer lorsqu’il n’y a pas d’obstacle et à changer de direction lorsqu’il rencontre un mur. Cela donne des trajectoires simples et efficaces.

La deuxième configuration (`[[ -1, -1],[ -1, 1],[ -1, 1]]`) est encore plus intéressante. Elle permet à l’agent d’avoir des trajectoires plus naturelles et moins robotiques, en alternant plus souvent entre les différentes actions. Elle permet aussi la capacité de l’agent à se débloquer contre les murs tout en explorant l’environnement de manière plus variée.





